In [1]:
%cd /kaggle/working

/kaggle/working


In [2]:
import numpy as np
import polars as pl

In [3]:
test_df = pl.read_parquet("input/test.parquet")
test_old_df = pl.read_parquet("input/test_old.parquet")

In [4]:
map_df = pl.read_parquet("output/preprocess/make_hack_map/base/map_hack.parquet")

In [5]:
map_df

location,pbuf_ozone_0,pbuf_ozone_1,pbuf_ozone_2,pbuf_ozone_3,pbuf_ozone_4,pbuf_ozone_5,pbuf_ozone_6,pbuf_ozone_7,pbuf_ozone_8,pbuf_ozone_9,month,day,seconds,tick
u32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,i64,i64,i64
0,2.4616e-7,4.4335e-7,7.9467e-7,0.000001,0.000002,0.000004,0.000007,0.000011,0.000013,0.000014,1,1,0,0
1,2.4163e-7,4.3519e-7,7.8003e-7,0.000001,0.000002,0.000004,0.000007,0.000011,0.000013,0.000014,1,1,0,0
2,2.6509e-7,4.7745e-7,8.5578e-7,0.000002,0.000003,0.000005,0.000008,0.000012,0.000014,0.000015,1,1,0,0
3,2.5930e-7,4.6701e-7,8.3707e-7,0.000001,0.000003,0.000004,0.000007,0.000012,0.000014,0.000015,1,1,0,0
4,2.3942e-7,4.3120e-7,7.7288e-7,0.000001,0.000002,0.000004,0.000007,0.000011,0.000012,0.000014,1,1,0,0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
379,2.9971e-7,5.3980e-7,9.6753e-7,0.000002,0.000003,0.000005,0.000009,0.000014,0.000013,0.000012,12,31,85200,71
380,3.0109e-7,5.4228e-7,9.7199e-7,0.000002,0.000003,0.000005,0.000009,0.000014,0.000013,0.000012,12,31,85200,71
381,3.0464e-7,5.4867e-7,9.8344e-7,0.000002,0.000003,0.000005,0.000009,0.000014,0.000013,0.000012,12,31,85200,71


In [5]:
for i in range(0, 10):
    col = f"pbuf_ozone_{i}"
    print(col, len(map_df[col].unique()))

pbuf_ozone_0 10091520
pbuf_ozone_1 10091520
pbuf_ozone_2 10091520
pbuf_ozone_3 10091520
pbuf_ozone_4 10091520
pbuf_ozone_5 10091520
pbuf_ozone_6 10091520
pbuf_ozone_7 10091520
pbuf_ozone_8 10091520
pbuf_ozone_9 10091520


In [6]:
skip = 10
ozeon_cols = [f"pbuf_ozone_{skip*s}" for s in range(60 // skip)]

test_info = test_df[["sample_id"] + ozeon_cols].join(
    map_df,
    on=ozeon_cols,
    how="left",
)
print("null", test_info["month"].is_null().sum())

test_info

null 625000


sample_id,pbuf_ozone_0,pbuf_ozone_10,pbuf_ozone_20,pbuf_ozone_30,pbuf_ozone_40,pbuf_ozone_50,location,month,day,seconds,tick
str,f64,f64,f64,f64,f64,f64,u32,i64,i64,i64,i64
"""test_0""",2.9057e-7,0.000015,8.9661e-8,7.1685e-8,7.3007e-8,7.0039e-8,null,null,null,null,null
"""test_10""",3.0539e-7,0.000013,7.8383e-7,1.0526e-7,9.2631e-8,8.2155e-8,null,null,null,null,null
"""test_100""",3.2143e-7,0.000012,4.8083e-7,6.5793e-8,5.5449e-8,4.8415e-8,null,null,null,null,null
"""test_1000""",3.0851e-7,0.000014,1.0223e-7,8.0955e-8,7.7514e-8,8.1617e-8,null,null,null,null,null
"""test_10000""",2.8595e-7,0.000015,8.7950e-8,6.0746e-8,4.8210e-8,3.8620e-8,null,null,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…
"""test_99994""",2.9806e-7,0.000014,6.8858e-8,8.3828e-8,8.4656e-8,6.9248e-8,null,null,null,null,null
"""test_99995""",2.9850e-7,0.000011,0.000002,1.3587e-7,8.5169e-8,7.6053e-8,null,null,null,null,null
"""test_99996""",2.4464e-7,0.000008,0.000002,1.8611e-7,9.6251e-8,7.6861e-8,null,null,null,null,null


In [25]:
ozeon_cols = [f"pbuf_ozone_{s}" for s in range(10)]

test_info = (
    test_df[["sample_id"] + ozeon_cols]
    .with_columns([pl.col(col).round(12) for col in ozeon_cols])
    .join(
        map_df.with_columns([pl.col(col).round(12) for col in ozeon_cols]),
        on=ozeon_cols,
        how="left",
    )
)
print("null", test_info["month"].is_null().sum())

test_info

null 0


sample_id,pbuf_ozone_0,pbuf_ozone_1,pbuf_ozone_2,pbuf_ozone_3,pbuf_ozone_4,pbuf_ozone_5,pbuf_ozone_6,pbuf_ozone_7,pbuf_ozone_8,pbuf_ozone_9,location,month,day,seconds,tick
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,u32,i64,i64,i64,i64
"""test_0""",2.9057e-7,5.2333e-7,9.3802e-7,0.000002,0.000003,0.000005,0.000008,0.000013,0.000015,0.000016,230,3,8,28800,24
"""test_10""",3.0539e-7,5.5003e-7,9.8587e-7,0.000002,0.000003,0.000005,0.000009,0.000014,0.000014,0.000014,55,3,14,0,0
"""test_100""",3.2143e-7,5.7890e-7,0.000001,0.000002,0.000003,0.000006,0.000009,0.000015,0.000014,0.000013,1,7,27,0,0
"""test_1000""",3.0851e-7,5.5565e-7,9.9595e-7,0.000002,0.000003,0.000005,0.000009,0.000014,0.000015,0.000015,25,7,24,57600,48
"""test_10000""",2.8595e-7,5.1502e-7,9.2312e-7,0.000002,0.000003,0.000005,0.000008,0.000013,0.000015,0.000016,216,3,18,57600,48
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""test_99994""",2.9806e-7,5.3682e-7,9.6220e-7,0.000002,0.000003,0.000005,0.000009,0.000014,0.000014,0.000015,106,1,25,28800,24
"""test_99995""",2.9850e-7,5.3761e-7,9.6361e-7,0.000002,0.000003,0.000005,0.000009,0.000013,0.000012,0.000011,356,12,30,0,0
"""test_99996""",2.4464e-7,4.4061e-7,7.8975e-7,0.000001,0.000002,0.000004,0.000007,0.000011,0.00001,0.000009,360,5,2,0,0


In [27]:
test_info['seconds'].value_counts()

seconds,count
i64,u32
57600,208128
0,208512
28800,208360


In [23]:
test_info["month"].value_counts().sort("month")

month,count
i64,u32
null,132005
1,30106
2,27066
3,50806
4,46756
…,…
8,54040
9,25163
10,24599


In [7]:
test_old_info = test_old_df[["sample_id", "pbuf_ozone_10"]].join(
    map_df, on="pbuf_ozone_10", how="left"
)

print("null", test_old_info["month"].is_null().sum())
test_old_info

null 0


sample_id,pbuf_ozone_10,location,pbuf_ozone_0,pbuf_ozone_20,pbuf_ozone_30,pbuf_ozone_40,pbuf_ozone_50,month,day,seconds,tick
str,f64,u32,f64,f64,f64,f64,f64,i64,i64,i64,i64
"""test_169651""",0.000013,0,2.6356e-7,2.4476e-7,6.7419e-8,5.5847e-8,3.4644e-8,3,1,0,0
"""test_524862""",0.000013,1,2.6128e-7,3.2513e-7,6.8855e-8,5.3563e-8,3.3092e-8,3,1,0,0
"""test_634129""",0.000015,2,2.7551e-7,1.4950e-7,7.0802e-8,5.8015e-8,2.9817e-8,3,1,0,0
"""test_403572""",0.000014,3,2.7210e-7,1.8384e-7,7.1740e-8,5.8858e-8,2.8088e-8,3,1,0,0
"""test_484578""",0.000013,4,2.6040e-7,3.5960e-7,6.9934e-8,5.2993e-8,3.2582e-8,3,1,0,0
…,…,…,…,…,…,…,…,…,…,…,…
"""test_578220""",0.000015,227,3.0087e-7,8.8404e-8,6.2248e-8,6.3355e-8,5.5999e-8,1,7,79200,66
"""test_395695""",0.000016,228,3.0920e-7,7.2369e-8,5.1252e-8,5.4098e-8,4.7274e-8,1,7,79200,66
"""test_88942""",0.000016,229,3.0892e-7,7.5348e-8,5.6648e-8,6.0303e-8,5.5206e-8,1,7,79200,66
